### Setup Environment & Download FER2013

In [ ]:
# Cell 1: Setup Environment & Download FER2013
print("🚀 CNN TRAINING - TAHAP 2 DIMULAI!")
print("="*50)

# Install dependencies
%pip install tensorflow opencv-python pandas numpy matplotlib seaborn scikit-learn
%pip install mtcnn pillow kaggle

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import os
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Check GPU availability
print(f"🔧 TensorFlow version: {tf.__version__}")
print(f"🖥️  GPU Available: {tf.config.list_physical_devices('GPU')}")
if tf.config.list_physical_devices('GPU'):
    print("✅ GPU detected! Training will be accelerated")
else:
    print("⚠️  No GPU detected. Training will use CPU (slower)")

# Set memory growth for GPU (if available)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU memory growth enabled")
    except RuntimeError as e:
        print(f"⚠️  GPU setup warning: {e}")

# Create directories
os.makedirs('data/fer2013', exist_ok=True)
os.makedirs('models', exist_ok=True)
os.makedirs('results/cnn', exist_ok=True)

print("✅ Environment setup complete!")
print("📁 Directories created: data/fer2013/, models/, results/cnn/")

### Load and Preprocess FER2013 Data

In [ ]:
# Cell 2: Load and Preprocess FER2013 Data
print("📊 LOADING & PREPROCESSING FER2013 DATA...")
print("="*50)

# Emotion labels (sesuai dengan Face API Anda)
emotion_labels = ['angry', 'disgusted', 'fearful', 'happy', 'sad', 'surprised', 'neutral']
num_classes = len(emotion_labels)

print(f"🎯 Target emotions: {emotion_labels}")
print(f"📊 Number of classes: {num_classes}")

# Data generators untuk training
def create_data_generators(batch_size=32, img_size=(48, 48)):
    """Create data generators for FER2013"""
    
    # Training data augmentation (seperti di paper)
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    
    # Validation data (no augmentation)
    val_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    # Load data
    try:
        # Split training data into train/validation (80/20)
        train_datagen_split = ImageDataGenerator(
            rescale=1./255,
            rotation_range=10,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.1,
            zoom_range=0.1,
            horizontal_flip=True,
            fill_mode='nearest',
            validation_split=0.2  # Split 20% for validation
        )
        
        # Training generator (80% of train data)
        train_generator = train_datagen_split.flow_from_directory(
            'data/fer2013/train',
            target_size=img_size,
            batch_size=batch_size,
            color_mode='grayscale',
            class_mode='categorical',
            shuffle=True,
            seed=42,
            subset='training'  # Use training subset
        )
        
        # Validation generator (20% of train data)
        validation_generator = train_datagen_split.flow_from_directory(
            'data/fer2013/train',  # Same directory as train
            target_size=img_size,
            batch_size=batch_size,
            color_mode='grayscale',
            class_mode='categorical',
            shuffle=False,
            seed=42,
            subset='validation'  # Use validation subset
        )
        
        test_generator = test_datagen.flow_from_directory(
            'data/fer2013/test',
            target_size=img_size,
            batch_size=batch_size,
            color_mode='grayscale',
            class_mode='categorical',
            shuffle=False,
            seed=42
        )
        
        print("✅ Data generators created successfully!")
        
        # Print dataset info
        print(f"\n📊 Dataset Information:")
        print(f"   Training samples: {train_generator.samples}")
        print(f"   Validation samples: {validation_generator.samples}")
        print(f"   Test samples: {test_generator.samples}")
        print(f"   Batch size: {batch_size}")
        print(f"   Image size: {img_size}")
        
        # Print class distribution
        print(f"\n🏷️  Class mapping:")
        for class_name, class_idx in train_generator.class_indices.items():
            print(f"   {class_idx}: {class_name}")
        
        return train_generator, validation_generator, test_generator
        
    except Exception as e:
        print(f"❌ Error creating data generators: {e}")
        print("💡 Make sure FER2013 dataset is properly downloaded and structured")
        return None, None, None

# Create data generators
print("\n🔄 Creating data generators...")
train_gen, val_gen, test_gen = create_data_generators(batch_size=32, img_size=(48, 48))

if train_gen is not None:
    # Visualize sample data
    print("\n🖼️  Visualizing sample training data...")
    
    # Get a batch of training data
    sample_batch_x, sample_batch_y = next(train_gen)
    
    # Plot sample images
    plt.figure(figsize=(12, 8))
    for i in range(min(12, len(sample_batch_x))):
        plt.subplot(3, 4, i+1)
        plt.imshow(sample_batch_x[i].reshape(48, 48), cmap='gray')
        
        # Get emotion label
        emotion_idx = np.argmax(sample_batch_y[i])
        emotion_name = emotion_labels[emotion_idx] if emotion_idx < len(emotion_labels) else f"Class_{emotion_idx}"
        
        plt.title(f'{emotion_name}')
        plt.axis('off')
    
    plt.suptitle('Sample Training Images from FER2013', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig('results/cnn/fer2013_samples.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Sample visualization saved to results/cnn/fer2013_samples.png")
    
    # Reset generator
    train_gen.reset()
    
    print("\n🎯 Data preprocessing complete!")
    print("📈 Ready for model building...")
    
else:
    print("❌ Data preprocessing failed!")
    print("🔧 Please check FER2013 dataset structure and try again")

print("\n🎯 NEXT: Run Cell 3 to build CNN architecture")

### Build CNN Architecture (Based on Paper)

In [ ]:
# Cell 3: Build CNN Architecture (Based on Paper)
print("🏗️  BUILDING CNN ARCHITECTURE...")
print("="*50)

def build_lightweight_cnn(input_shape=(48, 48, 1), num_classes=7):
    """
    Build lightweight CNN for facial emotion recognition
    Based on paper's approach with Adam robust optimization
    """
    
    model = Sequential([
        # First Convolutional Block
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Second Convolutional Block  
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Third Convolutional Block
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Flatten and Dense Layers
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        
        # Output layer
        Dense(num_classes, activation='softmax')
    ])
    
    return model

# Build model
print("🔨 Building lightweight CNN model...")
model = build_lightweight_cnn(input_shape=(48, 48, 1), num_classes=7)

# Compile with Adam optimizer (sesuai paper)
print("⚙️  Compiling model with Adam optimizer...")
model.compile(
    optimizer=Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Model summary
print("\n📋 MODEL ARCHITECTURE SUMMARY:")
model.summary()

# Calculate total parameters
total_params = model.count_params()
print(f"\n📊 Total Parameters: {total_params:,}")
print(f"🏋️  Model Size: ~{total_params * 4 / (1024*1024):.1f} MB (float32)")

# Visualize model architecture
print("\n🎨 Creating model visualization...")
try:
    tf.keras.utils.plot_model(
        model, 
        to_file='results/cnn/model_architecture.png',
        show_shapes=True,
        show_layer_names=True,
        rankdir='TB',
        dpi=300
    )
    print("✅ Model architecture saved to results/cnn/model_architecture.png")
except Exception as e:
    print(f"⚠️  Model visualization failed: {e}")

# Setup training callbacks
print("\n🔧 Setting up training callbacks...")

# Early stopping
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# Learning rate reduction
reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.2,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

# Model checkpoint
model_checkpoint = ModelCheckpoint(
    'models/best_fer_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

callbacks = [early_stopping, reduce_lr, model_checkpoint]

print("✅ Callbacks configured:")
print("   - Early stopping (patience=10)")
print("   - Learning rate reduction (factor=0.2, patience=5)")
print("   - Model checkpoint (save best weights)")

# Training configuration
EPOCHS = 100  # Paper menggunakan hingga 10,000 epochs, tapi kita batasi untuk speed
BATCH_SIZE = 32

print(f"\n🎯 TRAINING CONFIGURATION:")
print(f"   Epochs: {EPOCHS}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Optimizer: Adam (lr=0.001)")
print(f"   Loss: Categorical Crossentropy")

print("\n✅ Model architecture complete!")
print("🚀 Ready for training...")

print("\n🎯 NEXT: Run Cell 4 to start training the model")

# Save model configuration
model_config = {
    'architecture': 'Lightweight CNN',
    'input_shape': [48, 48, 1],
    'num_classes': 7,
    'total_parameters': int(total_params),
    'optimizer': 'Adam',
    'learning_rate': 0.001,
    'loss': 'categorical_crossentropy',
    'batch_size': BATCH_SIZE,
    'epochs': EPOCHS,
    'emotion_labels': emotion_labels
}

import json
with open('models/model_config.json', 'w') as f:
    json.dump(model_config, f, indent=2)

print("📁 Model configuration saved to models/model_config.json")

In [ ]:
# Cell 4: Execute Training
print("🚀 STARTING CNN TRAINING...")
print("="*50)

import time
from datetime import datetime

# Training start time
start_time = time.time()
print(f"⏰ Training started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Check if generators are available
if 'train_gen' not in globals() or train_gen is None:
    print("❌ Data generators not found! Please run Cell 2 first.")
else:
    print(f"📊 Training on {train_gen.samples} samples")
    print(f"📊 Validating on {val_gen.samples} samples")
    
    # Calculate steps per epoch
    steps_per_epoch = train_gen.samples // BATCH_SIZE
    validation_steps = val_gen.samples // BATCH_SIZE
    
    print(f"🔄 Steps per epoch: {steps_per_epoch}")
    print(f"🔄 Validation steps: {validation_steps}")
    
    print("\n🎯 Starting training process...")
    print("📈 Progress will be displayed below:")
    print("-" * 50)
    
    try:
        # Train the model (Fixed for TensorFlow compatibility)
        try:
            # Try with workers parameter (TensorFlow 2.4+)
            history = model.fit(
                train_gen,
                steps_per_epoch=steps_per_epoch,
                epochs=EPOCHS,
                validation_data=val_gen,
                validation_steps=validation_steps,
                callbacks=callbacks,
                verbose=1,
                workers=4,
                use_multiprocessing=True
            )
        except TypeError:
            # Fallback for older TensorFlow versions
            print("⚠️  Workers parameter not supported, using single-threaded training...")
            history = model.fit(
                train_gen,
                steps_per_epoch=steps_per_epoch,
                epochs=EPOCHS,
                validation_data=val_gen,
                validation_steps=validation_steps,
                callbacks=callbacks,
                verbose=1
            )
        
        # Training completed
        end_time = time.time()
        training_duration = end_time - start_time
        
        print("\n" + "="*50)
        print("🎉 TRAINING COMPLETED!")
        print(f"⏱️  Total training time: {training_duration/3600:.2f} hours")
        print(f"⏰ Completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        # Get best metrics
        best_epoch = np.argmax(history.history['val_accuracy'])
        best_val_acc = max(history.history['val_accuracy'])
        best_train_acc = history.history['accuracy'][best_epoch]
        
        print(f"\n📊 BEST RESULTS:")
        print(f"   Best epoch: {best_epoch + 1}")
        print(f"   Best validation accuracy: {best_val_acc:.4f}")
        print(f"   Training accuracy at best epoch: {best_train_acc:.4f}")
        
        # Save training history
        history_df = pd.DataFrame(history.history)
        history_df.to_csv('results/cnn/training_history.csv', index=False)
        print("💾 Training history saved to results/cnn/training_history.csv")
        
        # Save final model
        model.save('models/fer2013_trained_model.h5')
        print("💾 Final model saved to models/fer2013_trained_model.h5")
        
        print("\n✅ Training artifacts saved successfully!")
        
    except Exception as e:
        print(f"❌ Training failed with error: {e}")
        print("🔧 Please check GPU memory and data generators")
        
    except KeyboardInterrupt:
        print("\n⏸️  Training interrupted by user")
        print("💾 Saving current model state...")
        model.save('models/fer2013_interrupted_model.h5')
        print("✅ Model saved as fer2013_interrupted_model.h5")

print("\n🎯 NEXT: Run Cell 5 to visualize training results and evaluate model")

In [ ]:
# Cell 4B: Improved Training (OPTIONAL - jika mau improve accuracy)
print("🔧 IMPROVED CNN TRAINING...")
print("="*50)

# Load best model dari training sebelumnya
best_model = tf.keras.models.load_model('models/best_fer_model.h5')

# Analysis: Cek apakah overfitting atau underfitting
try:
    history_df = pd.read_csv('results/cnn/training_history.csv')
    
    final_train_acc = history_df['accuracy'].iloc[-1]
    final_val_acc = history_df['val_accuracy'].iloc[-1]
    best_val_acc = history_df['val_accuracy'].max()
    
    print(f"📊 TRAINING ANALYSIS:")
    print(f"   Final train accuracy: {final_train_acc:.4f}")
    print(f"   Final validation accuracy: {final_val_acc:.4f}")
    print(f"   Best validation accuracy: {best_val_acc:.4f}")
    print(f"   Gap (train-val): {final_train_acc - final_val_acc:.4f}")
    
    if final_train_acc - final_val_acc > 0.1:
        print("   🔍 Analysis: OVERFITTING detected")
        improvement_strategy = "reduce_overfitting"
    elif final_train_acc < 0.7:
        print("   🔍 Analysis: UNDERFITTING detected") 
        improvement_strategy = "increase_capacity"
    else:
        print("   🔍 Analysis: Model seems balanced")
        improvement_strategy = "fine_tune"
        
except:
    print("⚠️ Could not load training history")
    improvement_strategy = "fine_tune"

# Improvement strategies
if improvement_strategy == "reduce_overfitting":
    print("\n🎯 STRATEGY: Reduce Overfitting")
    
    # Add more regularization
    improved_model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1), padding='same'),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.4),  # Increased dropout
        
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.4),  # Increased dropout
        
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.5),  # More dropout
        
        Flatten(),
        Dense(256, activation='relu'),  # Reduced neurons
        BatchNormalization(),
        Dropout(0.6),  # High dropout
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(7, activation='softmax')
    ])
    
    # Lower learning rate
    improved_model.compile(
        optimizer=Adam(learning_rate=0.0005),  # Lower LR
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
elif improvement_strategy == "increase_capacity":
    print("\n🎯 STRATEGY: Increase Model Capacity")
    
    # Larger model
    improved_model = Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1), padding='same'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.3),
        
        Flatten(),
        Dense(1024, activation='relu'),  # Larger dense
        BatchNormalization(),
        Dropout(0.5),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(7, activation='softmax')
    ])
    
    # Higher learning rate
    improved_model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

else:
    print("\n🎯 STRATEGY: Continue with Current Model")
    print("✅ Model performance is acceptable for fine-tuning")
    improved_model = best_model

# Option to retrain
retrain = input("\n❓ Do you want to retrain with improved settings? (y/n): ").lower() == 'y'

if retrain and improvement_strategy != "fine_tune":
    print(f"\n🚀 Starting improved training...")
    
    # Enhanced callbacks
    improved_callbacks = [
        EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True, verbose=1),
        ReduceLROnPlateau(monitor='val_accuracy', factor=0.3, patience=10, min_lr=1e-8, verbose=1),
        ModelCheckpoint('models/improved_fer_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
    ]
    
    # Train improved model
    improved_history = improved_model.fit(
        train_gen,
        steps_per_epoch=train_gen.samples // 32,
        epochs=50,  # Fewer epochs but with better monitoring
        validation_data=val_gen,
        validation_steps=val_gen.samples // 32,
        callbacks=improved_callbacks,
        verbose=1
    )
    
    # Evaluate improved model
    improved_val_acc = max(improved_history.history['val_accuracy'])
    print(f"\n📊 IMPROVED RESULTS:")
    print(f"   Original validation accuracy: {best_val_acc:.4f}")
    print(f"   Improved validation accuracy: {improved_val_acc:.4f}")
    print(f"   Improvement: {improved_val_acc - best_val_acc:+.4f}")
    
    if improved_val_acc > best_val_acc:
        print("✅ Improvement successful! Using improved model.")
        # Save as best model
        improved_model.save('models/best_fer_model.h5')
    else:
        print("⚠️ No significant improvement. Keeping original model.")

else:
    print("✅ Continuing with current model (63% accuracy)")
    print("💡 Fine-tuning will likely improve performance significantly")

print(f"\n🎯 RECOMMENDATION:")
print(f"   📈 63% accuracy is acceptable for base model")
print(f"   🔧 Fine-tuning with your data should reach 70%+ easily")
print(f"   ⭐ Paper's final accuracy includes fine-tuning phase")

print(f"\n🎯 NEXT: Continue to Cell 5 for evaluation, then fine-tuning")

### Model Evaluation & Visualization

In [ ]:
# Cell 5: Model Evaluation & Visualization
print("📊 MODEL EVALUATION & VISUALIZATION...")
print("="*50)

# Load best model
try:
    best_model = tf.keras.models.load_model('models/best_fer_model.h5')
    print("✅ Best model loaded successfully!")
except:
    print("⚠️  Best model not found, using current model")
    best_model = model

# Plot training history
def plot_training_history(history):
    """Plot training and validation metrics"""
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Accuracy plot
    axes[0].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
    axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
    axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Loss plot
    axes[1].plot(history.history['loss'], label='Training Loss', linewidth=2)
    axes[1].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('results/cnn/training_curves.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return fig

# Plot training curves
if 'history' in globals():
    print("📈 Plotting training curves...")
    plot_training_history(history)
    print("✅ Training curves saved to results/cnn/training_curves.png")

# Evaluate on test set
print("\n🧪 Evaluating on test set...")
try:
    test_loss, test_accuracy = best_model.evaluate(test_gen, verbose=1)
    print(f"\n📊 TEST RESULTS:")
    print(f"   Test Loss: {test_loss:.4f}")
    print(f"   Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
    
    # Compare with paper results
    paper_accuracy = 0.73  # 73% from paper
    print(f"\n📋 COMPARISON WITH PAPER:")
    print(f"   Paper accuracy: {paper_accuracy:.4f} ({paper_accuracy*100:.2f}%)")
    print(f"   Our accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
    if test_accuracy >= paper_accuracy:
        print("   🎉 We achieved better accuracy than the paper!")
    else:
        print("   📈 Room for improvement, but good starting point")
        
except Exception as e:
    print(f"❌ Test evaluation failed: {e}")

# Generate predictions for confusion matrix
print("\n🔄 Generating predictions for confusion matrix...")
try:
    # Reset test generator
    test_gen.reset()
    
    # Get predictions
    predictions = best_model.predict(test_gen, verbose=1)
    predicted_classes = np.argmax(predictions, axis=1)
    
    # Get true labels
    true_classes = test_gen.classes
    
    # Generate confusion matrix
    cm = confusion_matrix(true_classes, predicted_classes)
    
    # Plot confusion matrix (style seperti paper)
    plt.figure(figsize=(10, 8))
    
    # Create heatmap
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=emotion_labels, yticklabels=emotion_labels,
                cbar_kws={'label': 'Number of Samples'})
    
    plt.title('Confusion Matrix for Emotion Detection\n(Similar to Paper Figure 12)', 
              fontsize=16, fontweight='bold', pad=20)
    plt.xlabel('Predicted Emotion Classes', fontsize=12)
    plt.ylabel('Actual Emotion Classes', fontsize=12)
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    
    plt.tight_layout()
    plt.savefig('results/cnn/confusion_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Confusion matrix saved to results/cnn/confusion_matrix.png")
    
    # Classification report
    print("\n📋 DETAILED CLASSIFICATION REPORT:")
    class_report = classification_report(true_classes, predicted_classes, 
                                       target_names=emotion_labels, 
                                       output_dict=True)
    
    # Format like paper's Table 2
    print(f"{'Class':<12} {'Precision (%)':<12} {'Recall (%)':<12} {'F1-score (%)':<12}")
    print("-" * 50)
    
    for i, emotion in enumerate(emotion_labels):
        precision = class_report[emotion]['precision'] * 100
        recall = class_report[emotion]['recall'] * 100
        f1 = class_report[emotion]['f1-score'] * 100
        print(f"{emotion:<12} {precision:<12.0f} {recall:<12.0f} {f1:<12.0f}")
    
    # Overall metrics
    accuracy = class_report['accuracy'] * 100
    weighted_avg = class_report['weighted avg']
    
    print("-" * 50)
    print(f"{'Weighted avg':<12} {weighted_avg['precision']*100:<12.0f} {weighted_avg['recall']*100:<12.0f} {weighted_avg['f1-score']*100:<12.0f}")
    print(f"Accuracy (%): {accuracy:.0f}")
    
    # Save classification report
    class_report_df = pd.DataFrame(class_report).transpose()
    class_report_df.to_csv('results/cnn/classification_report.csv')
    print("\n💾 Classification report saved to results/cnn/classification_report.csv")
    
except Exception as e:
    print(f"❌ Confusion matrix generation failed: {e}")

# Model performance summary
print("\n" + "="*60)
print("📋 CNN TRAINING SUMMARY REPORT")
print("="*60)

if 'history' in globals() and 'test_accuracy' in globals():
    print(f"🏗️  Model Architecture: Lightweight CNN")
    print(f"📊 Total Parameters: {model.count_params():,}")
    print(f"⏱️  Training Time: {training_duration/3600:.2f} hours")
    print(f"🎯 Best Validation Accuracy: {best_val_acc:.4f} ({best_val_acc*100:.2f}%)")
    print(f"🧪 Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
    print(f"📋 Paper Comparison: {'✅ Better' if test_accuracy >= 0.73 else '📈 Can improve'}")

print(f"\n💾 SAVED ARTIFACTS:")
print(f"   📁 models/best_fer_model.h5 - Best trained model")
print(f"   📁 results/cnn/training_curves.png - Training visualization")
print(f"   📁 results/cnn/confusion_matrix.png - Performance matrix")
print(f"   📁 results/cnn/classification_report.csv - Detailed metrics")

print(f"\n🎉 CNN TRAINING PHASE COMPLETE!")
print(f"🎯 NEXT PHASE: Fine-tuning with your video data")

print("\n🎯 NEXT: Run Cell 6 to prepare for fine-tuning with your data")

### Model Evaluation & Visualization

In [ ]:
# Cell 6: Complete Face Extraction (ALL FUNCTIONS INCLUDED)
print("🎥 PREPARING YOUR VIDEO DATA FOR FINE-TUNING...")
print("Format: user{ID}_challenge{N}_time{HHMMSS}.jpg")
print("="*60)

# Install and import all required libraries
try:
    from mtcnn import MTCNN
    print("✅ MTCNN imported successfully")
except ImportError:
    print("📦 Installing MTCNN...")
    %pip install mtcnn
    from mtcnn import MTCNN
    print("✅ MTCNN installed and imported")

import cv2
import os
import numpy as np
from PIL import Image
import pandas as pd
import re
from datetime import datetime, timedelta

# Initialize MTCNN detector globally
detector = MTCNN()
print("✅ MTCNN detector initialized")

# ============================================================================
# ALL REQUIRED FUNCTIONS
# ============================================================================

def extract_face_from_frame(image_path, target_size=(48, 48)):
    """
    Extract and preprocess face from video frame
    Similar to paper's MTCNN approach
    """
    try:
        # Read image
        img = cv2.imread(image_path)
        if img is None:
            return None
            
        # Convert BGR to RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Detect faces
        results = detector.detect_faces(img_rgb)
        
        if len(results) > 0:
            # Get the largest face (highest confidence)
            best_face = max(results, key=lambda x: x['confidence'])
            
            if best_face['confidence'] > 0.7:  # Confidence threshold
                # Extract face coordinates
                x, y, w, h = best_face['box']
                
                # Add padding
                padding = 20
                x = max(0, x - padding)
                y = max(0, y - padding)
                w = min(img_rgb.shape[1] - x, w + 2*padding)
                h = min(img_rgb.shape[0] - y, h + 2*padding)
                
                # Crop face
                face = img_rgb[y:y+h, x:x+w]
                
                # Resize to target size
                face_resized = cv2.resize(face, target_size)
                
                # Convert to grayscale (sesuai dengan training)
                face_gray = cv2.cvtColor(face_resized, cv2.COLOR_RGB2GRAY)
                
                # Normalize
                face_normalized = face_gray.astype('float32') / 255.0
                
                return face_normalized
        
        return None
        
    except Exception as e:
        print(f"⚠️ Error processing {image_path}: {e}")
        return None

def parse_your_filename(filename):
    """
    Parse filename dengan format: user{ID}_challenge{N}_time{HHMMSS}.jpg
    """
    
    # Pattern untuk format Anda
    pattern = r'user(\d+)_challenge(\d+)_time(\d{6})\.jpg'
    match = re.match(pattern, filename)
    
    if match:
        user_id = int(match.group(1))
        challenge = int(match.group(2))
        time_str = match.group(3)  # HHMMSS format
        
        # Convert HHMMSS to seconds for easier matching
        hours = int(time_str[:2])
        minutes = int(time_str[2:4])
        seconds = int(time_str[4:6])
        time_in_seconds = hours * 3600 + minutes * 60 + seconds
        
        return {
            'user_id': user_id,
            'challenge': challenge,
            'time': time_str,
            'time_seconds': time_in_seconds,
            'formatted_time': f"{hours:02d}:{minutes:02d}:{seconds:02d}"
        }
    
    return None

def map_frames_to_excel(video_directory, excel_data):
    """
    Map video frames ke Excel data berdasarkan user_id, challenge, dan waktu
    """
    
    print("🔍 MAPPING VIDEO FRAMES TO EXCEL DATA...")
    print("="*50)
    
    # Get all image files
    image_files = []
    for root, dirs, files in os.walk(video_directory):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                full_path = os.path.join(root, file)
                parsed = parse_your_filename(file)
                if parsed:
                    parsed['filename'] = file
                    parsed['filepath'] = full_path
                    image_files.append(parsed)
                else:
                    print(f"⚠️ Could not parse: {file}")
    
    print(f"📁 Found {len(image_files)} valid image files")
    
    # Convert to DataFrame
    frames_df = pd.DataFrame(image_files)
    
    if len(frames_df) == 0:
        print("❌ No valid frames found!")
        return None
    
    # Display frame distribution
    print(f"\n📊 FRAME DISTRIBUTION:")
    print(f"   Users: {frames_df['user_id'].nunique()}")
    print(f"   Challenges: {frames_df['challenge'].nunique()}")
    print(f"   Total frames: {len(frames_df)}")
    
    # Map challenge from page column
    def extract_challenge_from_page(page):
        """Extract challenge number from page URL"""
        if pd.isna(page):
            return None
        
        page_str = str(page).lower()
        if 'penjumlahan' in page_str or 'addition' in page_str:
            return 1
        elif 'perkalian' in page_str or 'multiplication' in page_str:
            return 2
        
        return 1  # Default to challenge 1

    excel_df = excel_data.copy()
    excel_df['challenge'] = excel_df['page'].apply(extract_challenge_from_page)
    
    # Simple mapping strategy
    mappings = []
    
    for _, frame_row in frames_df.iterrows():
        # Find matching Excel records
        matching_excel = excel_df[
            (excel_df['user_id'] == frame_row['user_id']) &
            (excel_df['challenge'] == frame_row['challenge'])
        ]
        
        if len(matching_excel) > 0:
            # Take first match for simplicity
            excel_row = matching_excel.iloc[0]
            mappings.append({
                'filename': frame_row['filename'],
                'filepath': frame_row['filepath'],
                'user_id': frame_row['user_id'],
                'challenge': frame_row['challenge'],
                'frame_time': frame_row['formatted_time'],
                'excel_idx': excel_row.name,
                'engagement_score': excel_row['engagement_score'],
                'engagement_level': excel_row['engagement_level'],
                'face_api_emotions': [
                    excel_row['neutral'], excel_row['happy'], excel_row['sad'],
                    excel_row['angry'], excel_row['fearful'], 
                    excel_row['disgusted'], excel_row['surprised']
                ]
            })
    
    # Convert to DataFrame
    mapping_df = pd.DataFrame(mappings)
    
    print(f"\n✅ MAPPING COMPLETE:")
    print(f"   Total frames: {len(frames_df)}")
    print(f"   Successfully mapped: {len(mapping_df)}")
    if len(frames_df) > 0:
        print(f"   Mapping rate: {len(mapping_df)/len(frames_df)*100:.1f}%")
    
    return mapping_df

# ============================================================================
# MAIN EXECUTION
# ============================================================================

# Load your processed engagement data dari Tahap 1
print("\n📊 Loading your engagement data from Tahap 1...")
try:
    your_data = pd.read_csv('results/processed_engagement_data.csv')
    print(f"✅ Loaded {len(your_data)} records from {your_data['user_id'].nunique()} users")
    
    # Display summary
    print(f"\n📋 Your Data Summary:")
    print(f"   Users: {your_data['user_id'].nunique()}")
    print(f"   Records: {len(your_data)}")
    print(f"   Unique pages: {your_data['page'].nunique()}")
    print(f"   Engagement distribution:")
    for level, count in your_data['engagement_level'].value_counts().items():
        print(f"     {level}: {count}")
        
except Exception as e:
    print(f"❌ Error loading your data: {e}")
    print("💡 Make sure you completed Tahap 1 (baseline analysis)")
    # Create dummy data for testing
    your_data = pd.DataFrame({
        'user_id': [97, 97, 98, 98],
        'page': ['/tantangan/penjumlahan', '/tantangan/perkalian', '/tantangan/penjumlahan', '/tantangan/perkalian'],
        'engagement_score': [0.12, 0.08, 0.15, 0.11],
        'engagement_level': ['Engaged', 'Disengaged', 'Highly Engaged', 'Engaged'],
        'neutral': [0.7, 0.8, 0.6, 0.75],
        'happy': [0.2, 0.1, 0.3, 0.15],
        'sad': [0.05, 0.08, 0.04, 0.06],
        'angry': [0.02, 0.01, 0.03, 0.02],
        'fearful': [0.01, 0.005, 0.02, 0.01],
        'disgusted': [0.01, 0.003, 0.01, 0.005],
        'surprised': [0.01, 0.002, 0.005, 0.005]
    })
    print("⚠️ Using dummy data for testing")

# Setup directories
video_frames_dir = "data/video_frames"  # SESUAIKAN dengan lokasi frame video Anda
face_crops_dir = "data/face_crops"

print(f"\n🎯 CONFIGURATION:")
print(f"   Video frames directory: {video_frames_dir}")
print(f"   Face crops output: {face_crops_dir}")

# Create face crops directory
os.makedirs(face_crops_dir, exist_ok=True)

# Map your frames to Excel data
if os.path.exists(video_frames_dir):
    print(f"\n🔍 Mapping video frames to Excel data...")
    
    # Use custom mapping function
    frame_mapping = map_frames_to_excel(video_frames_dir, your_data)
    
    if frame_mapping is not None and len(frame_mapping) > 0:
        print(f"\n📊 MAPPING RESULTS:")
        print(f"   Successfully mapped: {len(frame_mapping)} frames")
        print(f"   Users covered: {frame_mapping['user_id'].nunique()}")
        print(f"   Challenges covered: {frame_mapping['challenge'].nunique()}")
        
        # Save mapping results
        frame_mapping.to_csv('data/frame_mapping.csv', index=False)
        print(f"💾 Mapping saved to data/frame_mapping.csv")
        
        # Extract faces from mapped frames
        print(f"\n🎯 Extracting faces from video frames...")
        
        face_data = []
        processed_count = 0
        
        for idx, row in frame_mapping.iterrows():
            if os.path.exists(row['filepath']):
                # Extract face using MTCNN
                face = extract_face_from_frame(row['filepath'])
                
                if face is not None:
                    # Save processed face
                    face_filename = f"face_{processed_count:06d}_{row['user_id']}_c{row['challenge']}.jpg"
                    face_path = os.path.join(face_crops_dir, face_filename)
                    
                    # Save as image
                    face_img = (face * 255).astype(np.uint8)
                    cv2.imwrite(face_path, face_img)
                    
                    # Store metadata
                    face_data.append({
                        'face_path': face_path,
                        'original_frame': row['filepath'],
                        'user_id': row['user_id'],
                        'challenge': row['challenge'],
                        'frame_time': row['frame_time'],
                        'engagement_score': row['engagement_score'],
                        'engagement_level': row['engagement_level'],
                        'face_api_emotions': row['face_api_emotions']
                    })
                    
                    processed_count += 1
                    
                    if processed_count % 20 == 0:
                        print(f"   Processed {processed_count} faces...")
                else:
                    print(f"   ⚠️ No face detected in {row['filename']}")
        
        # Convert to DataFrame
        face_data_df = pd.DataFrame(face_data)
        
        print(f"\n✅ Face extraction complete!")
        print(f"   Total faces extracted: {len(face_data_df)}")
        
        if len(face_data_df) > 0:
            print(f"   Users: {face_data_df['user_id'].nunique()}")
            print(f"   Challenges: {face_data_df['challenge'].nunique()}")
            
            # Engagement distribution in extracted faces
            print(f"\n🎯 Engagement distribution in extracted faces:")
            for level, count in face_data_df['engagement_level'].value_counts().items():
                percentage = (count / len(face_data_df)) * 100
                print(f"   {level}: {count} ({percentage:.1f}%)")
            
            # Save face data metadata
            face_data_df.to_csv('data/face_data_metadata.csv', index=False)
            print(f"💾 Face metadata saved to data/face_data_metadata.csv")
            
            # Show sample mappings
            print(f"\n📋 SAMPLE MAPPINGS:")
            sample_data = frame_mapping.head(3)[['filename', 'user_id', 'challenge', 'frame_time', 'engagement_level']]
            print(sample_data.to_string(index=False))
        else:
            print("❌ No faces were successfully extracted")
            print("💡 Check if:")
            print("   - Images contain clear faces")
            print("   - Face detection confidence threshold (currently 0.7)")
            print("   - Image quality and lighting")
        
    else:
        print(f"❌ No frames could be mapped to Excel data")
        print(f"💡 Check if:")
        print(f"   - Filenames follow format: user{ID}_challenge{N}_time{HHMMSS}.jpg")
        print(f"   - User IDs match those in Excel data")
        print(f"   - Challenge mapping is correct")
        
else:
    print(f"❌ Video frames directory not found: {video_frames_dir}")
    print(f"💡 Please create the directory and add your video frames")
    
    # Instructions for your specific format
    print(f"\n📋 EXPECTED FILE FORMAT:")
    print(f"   user97_challenge1_time115444.jpg")
    print(f"   user97_challenge1_time115449.jpg") 
    print(f"   user97_challenge2_time120530.jpg")
    print(f"   user98_challenge1_time091245.jpg")
    print(f"   ...")

print(f"\n🎯 NEXT: Run Cell 7 to fine-tune the pre-trained model")

### Fine-tune Pre-trained Model with Your Data

In [ ]:
# Cell 7: Fine-tune Pre-trained Model (FIXED - Sequential Compatible)
print("🔧 FINE-TUNING PRE-TRAINED MODEL...")
print("="*60)

# Load pre-trained model
print("📦 Loading pre-trained FER2013 model...")
try:
    pretrained_model = tf.keras.models.load_model('models/best_fer_model.h5')
    print("✅ Pre-trained model loaded successfully!")
    print(f"📊 Model type: {type(pretrained_model)}")
    
    # Build the model to initialize input shape
    if hasattr(pretrained_model, 'build') and not pretrained_model.built:
        pretrained_model.build(input_shape=(None, 48, 48, 1))
        print("✅ Model built with input shape")
    
except Exception as e:
    print(f"❌ Error loading pre-trained model: {e}")
    print("💡 Make sure you completed CNN training in previous steps")

# Load your face data
try:
    face_metadata = pd.read_csv('data/face_data_metadata.csv')
    print(f"✅ Loaded face metadata: {len(face_metadata)} faces")
    print(f"   Users: {face_metadata['user_id'].nunique()}")
    print(f"   Challenges: {face_metadata['challenge'].nunique()}")
except Exception as e:
    print(f"❌ Error loading face metadata: {e}")
    print("💡 Make sure you completed face extraction in Cell 6")
    # Create dummy data for testing
    face_metadata = pd.DataFrame({
        'face_path': ['data/face_crops/face_000001_97_c1.jpg'],
        'user_id': [97],
        'engagement_score': [0.12],
        'engagement_level': ['Engaged'],
        'face_api_emotions': [[[0.7, 0.2, 0.05, 0.02, 0.01, 0.01, 0.01]]]
    })
    print("⚠️ Using dummy data for testing")

def create_fine_tuning_model_fixed(pretrained_model, num_classes=7):
    """
    Create fine-tuning model from pre-trained CNN (Fixed for Sequential)
    """
    
    print("🔨 Creating fine-tuning model...")
    
    # Method 1: Clone and modify existing model
    try:
        # Get model config and weights
        config = pretrained_model.get_config()
        weights = pretrained_model.get_weights()
        
        # Create new model with same architecture
        new_model = tf.keras.Sequential.from_config(config)
        new_model.set_weights(weights)
        
        # Remove last few layers
        while len(new_model.layers) > 0 and len(new_model.layers) >= 3:
            new_model.pop()  # Remove last layer
        
        # Add new classification head for fine-tuning
        new_model.add(Dense(128, activation='relu', name='fine_tune_dense1'))
        new_model.add(Dropout(0.5, name='fine_tune_dropout1'))
        new_model.add(Dense(64, activation='relu', name='fine_tune_dense2'))
        new_model.add(Dropout(0.3, name='fine_tune_dropout2'))
        new_model.add(Dense(num_classes, activation='softmax', name='fine_tune_output'))
        
        # Freeze early layers
        for i, layer in enumerate(new_model.layers[:-4]):
            layer.trainable = False
            
        print(f"✅ Fine-tuning model created using model cloning")
        return new_model
        
    except Exception as e:
        print(f"⚠️ Model cloning failed: {e}")
        
    # Method 2: Create new model and transfer weights layer by layer
    try:
        print("🔄 Trying weight transfer method...")
        
        # Create new model with similar architecture
        new_model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1), padding='same'),
            BatchNormalization(),
            Conv2D(32, (3, 3), activation='relu', padding='same'),
            MaxPooling2D(pool_size=(2, 2)),
            Dropout(0.25),
            
            Conv2D(64, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            Conv2D(64, (3, 3), activation='relu', padding='same'),
            MaxPooling2D(pool_size=(2, 2)),
            Dropout(0.25),
            
            Conv2D(128, (3, 3), activation='relu', padding='same'),
            BatchNormalization(),
            Conv2D(128, (3, 3), activation='relu', padding='same'),
            MaxPooling2D(pool_size=(2, 2)),
            Dropout(0.25),
            
            Flatten(),
            # Modified dense layers for fine-tuning
            Dense(256, activation='relu'),  # Smaller than original
            BatchNormalization(),
            Dropout(0.5),
            Dense(128, activation='relu'),
            Dropout(0.5),
            Dense(num_classes, activation='softmax')
        ])
        
        # Build the model
        new_model.build(input_shape=(None, 48, 48, 1))
        
        # Transfer weights from pretrained model (compatible layers only)
        pretrained_weights = pretrained_model.get_weights()
        new_weights = new_model.get_weights()
        
        # Transfer as many weights as possible
        min_layers = min(len(pretrained_weights), len(new_weights) - 4)  # Keep last 4 layers random
        
        for i in range(min_layers):
            if pretrained_weights[i].shape == new_weights[i].shape:
                new_weights[i] = pretrained_weights[i]
        
        new_model.set_weights(new_weights)
        
        # Freeze early layers
        for layer in new_model.layers[:-4]:
            layer.trainable = False
            
        print(f"✅ Fine-tuning model created using weight transfer")
        return new_model
        
    except Exception as e:
        print(f"⚠️ Weight transfer failed: {e}")
    
    # Method 3: Use pretrained model as-is with modified training
    print("🔄 Using pretrained model as-is for fine-tuning...")
    
    # Freeze early layers of pretrained model
    for i, layer in enumerate(pretrained_model.layers[:-3]):
        layer.trainable = False
    
    return pretrained_model

# Create fine-tuning model
print("\n🏗️ Creating fine-tuning model...")
fine_tune_model = create_fine_tuning_model_fixed(pretrained_model)

# Compile with lower learning rate for fine-tuning
fine_tune_model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Lower LR for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ Fine-tuning model created and compiled!")
print(f"📊 Trainable parameters: {sum([tf.keras.backend.count_params(w) for w in fine_tune_model.trainable_weights]):,}")
print(f"📊 Total parameters: {fine_tune_model.count_params():,}")

# Prepare data generators for your data
def create_your_data_generators_simple(face_metadata, face_crops_dir, batch_size=16):
    """
    Create data generators from your extracted faces (SIMPLIFIED)
    Using Face API emotions as pseudo-labels
    """
    
    print("📊 Preparing your face data...")
    
    # Prepare data arrays
    face_paths = []
    emotion_labels = []
    engagement_labels = []
    
    for idx, row in face_metadata.iterrows():
        if os.path.exists(row['face_path']):
            face_paths.append(row['face_path'])
            
            # Use Face API emotions as labels
            try:
                if isinstance(row['face_api_emotions'], str):
                    emotions = eval(row['face_api_emotions'])
                else:
                    emotions = row['face_api_emotions']
                
                # Ensure it's a list/array
                if isinstance(emotions, (list, np.ndarray)) and len(emotions) >= 7:
                    emotion_labels.append(emotions[:7])  # Take first 7 emotions
                else:
                    # Default neutral emotion
                    emotion_labels.append([1, 0, 0, 0, 0, 0, 0])
                    
            except:
                # Default neutral emotion if parsing fails
                emotion_labels.append([1, 0, 0, 0, 0, 0, 0])
                
            engagement_labels.append(row['engagement_level'])
    
    print(f"📊 Prepared {len(face_paths)} faces for training")
    
    if len(face_paths) == 0:
        print("❌ No valid face data found!")
        return None, None, None, None
    
    # Convert to numpy arrays
    X = np.zeros((len(face_paths), 48, 48, 1))
    y = np.array(emotion_labels)
    
    # Load and preprocess images
    valid_indices = []
    for i, face_path in enumerate(face_paths):
        try:
            img = cv2.imread(face_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, (48, 48))
                X[i] = img.reshape(48, 48, 1) / 255.0
                valid_indices.append(i)
        except Exception as e:
            print(f"⚠️ Error loading {face_path}: {e}")
    
    # Keep only valid data
    if len(valid_indices) == 0:
        print("❌ No valid images could be loaded!")
        return None, None, None, None
        
    X = X[valid_indices]
    y = y[valid_indices]
    
    print(f"✅ Data arrays created: X{X.shape}, y{y.shape}")
    
    # Split data (80% train, 20% validation)
    if len(X) >= 5:  # Need minimum data for split
        from sklearn.model_selection import train_test_split
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
    else:
        # Too little data, use all for training
        X_train, X_val = X, X
        y_train, y_val = y, y
        print("⚠️ Limited data: using same data for train and validation")
    
    print(f"📊 Train: {X_train.shape[0]} samples")
    print(f"📊 Validation: {X_val.shape[0]} samples")
    
    return X_train, X_val, y_train, y_val

# Create your data generators
if len(face_metadata) > 0:
    print("\n📊 Creating data generators from your faces...")
    X_train, X_val, y_train, y_val = create_your_data_generators_simple(
        face_metadata, face_crops_dir, batch_size=16
    )
    
    if X_train is not None:
        # Data augmentation for your data
        your_datagen = ImageDataGenerator(
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.1,
            zoom_range=0.1,
            horizontal_flip=True,
            fill_mode='nearest'
        )
        
        # Setup callbacks for fine-tuning
        fine_tune_callbacks = [
            EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=8, min_lr=1e-8),
            ModelCheckpoint('models/fine_tuned_model.h5', monitor='val_accuracy', 
                           save_best_only=True, verbose=1)
        ]
        
        # Fine-tuning training
        print("\n🚀 Starting fine-tuning...")
        print("⚙️ Using lower learning rate and smaller batch size")
        print("🎯 Training on your programming context data...")
        
        try:
            fine_tune_history = fine_tune_model.fit(
                your_datagen.flow(X_train, y_train, batch_size=8),  # Smaller batch size
                steps_per_epoch=max(1, len(X_train) // 8),
                epochs=30,  # Fewer epochs for fine-tuning
                validation_data=(X_val, y_val),
                callbacks=fine_tune_callbacks,
                verbose=1
            )
            
            print("\n🎉 Fine-tuning completed!")
            
            # Evaluate fine-tuned model
            print("\n📊 Evaluating fine-tuned model...")
            val_loss, val_accuracy = fine_tune_model.evaluate(X_val, y_val, verbose=0)
            print(f"   Validation accuracy: {val_accuracy:.4f} ({val_accuracy*100:.2f}%)")
            
            # Save fine-tuning results
            fine_tune_history_df = pd.DataFrame(fine_tune_history.history)
            fine_tune_history_df.to_csv('results/fine_tune_history.csv', index=False)
            
            print("💾 Fine-tuning history saved to results/fine_tune_history.csv")
            
            # Quick visualization
            plt.figure(figsize=(12, 4))
            
            plt.subplot(1, 2, 1)
            plt.plot(fine_tune_history.history['accuracy'], label='Training')
            plt.plot(fine_tune_history.history['val_accuracy'], label='Validation')
            plt.title('Fine-tuning Accuracy')
            plt.legend()
            
            plt.subplot(1, 2, 2)
            plt.plot(fine_tune_history.history['loss'], label='Training')
            plt.plot(fine_tune_history.history['val_loss'], label='Validation')
            plt.title('Fine-tuning Loss')
            plt.legend()
            
            plt.tight_layout()
            plt.savefig('results/fine_tune_curves.png', dpi=300, bbox_inches='tight')
            plt.show()
            
            print("✅ Fine-tuning visualization saved to results/fine_tune_curves.png")
            
        except Exception as e:
            print(f"❌ Fine-tuning failed: {e}")
            print("💡 This might be due to limited data or model compatibility")
            print("   Consider using the pre-trained model as-is")
            
    else:
        print("❌ Could not prepare training data")
        
else:
    print("❌ No face data available for fine-tuning")
    print("💡 Please complete face extraction in Cell 6 first")

print("\n🎯 NEXT: Run Cell 8 for hybrid analysis (Face API + Fine-tuned CNN)")

### Hybrid Analysis - Face API + Fine-tuned CNN

In [ ]:
# Cell 8: Hybrid Analysis - Face API + CNN (FIXED - Complete)
print("🔄 HYBRID ANALYSIS: Face API + Fine-tuned CNN...")
print("="*70)

# Load models and data with error handling
print("📦 Loading models and data...")

# Load baseline data (from Tahap 1)
try:
    baseline_data = pd.read_csv('results/processed_engagement_data.csv')
    print(f"✅ Baseline data loaded: {len(baseline_data)} records")
except Exception as e:
    print(f"❌ Error loading baseline data: {e}")
    print("💡 Using dummy baseline data for testing")
    baseline_data = pd.DataFrame({
        'user_id': [97, 97, 98, 98, 99, 99],
        'page': ['/tantangan/penjumlahan'] * 6,
        'engagement_score': [0.12, 0.08, 0.15, 0.11, 0.09, 0.16],
        'engagement_level': ['Engaged', 'Disengaged', 'Highly Engaged', 'Engaged', 'Disengaged', 'Highly Engaged'],
        'neutral': [0.7, 0.8, 0.6, 0.75, 0.65, 0.55],
        'happy': [0.2, 0.1, 0.3, 0.15, 0.25, 0.35],
        'sad': [0.05, 0.08, 0.04, 0.06, 0.07, 0.03],
        'angry': [0.02, 0.01, 0.03, 0.02, 0.01, 0.04],
        'fearful': [0.01, 0.005, 0.02, 0.01, 0.015, 0.025],
        'disgusted': [0.01, 0.003, 0.01, 0.005, 0.008, 0.015],
        'surprised': [0.01, 0.002, 0.005, 0.005, 0.007, 0.02]
    })

# Load models with fallbacks
original_model = None
fine_tuned_model = None

# Try to load fine-tuned model
try:
    fine_tuned_model = tf.keras.models.load_model('models/fine_tuned_model.h5')
    print("✅ Fine-tuned model loaded successfully")
except Exception as e:
    print(f"⚠️ Fine-tuned model not found: {e}")
    
    # Try to load original pre-trained model as fallback
    try:
        fine_tuned_model = tf.keras.models.load_model('models/best_fer_model.h5')
        print("✅ Using pre-trained model as fallback")
    except Exception as e2:
        print(f"❌ No models available: {e2}")
        fine_tuned_model = None

# Load face metadata
face_metadata = None
try:
    if os.path.exists('data/face_data_metadata.csv'):
        face_metadata = pd.read_csv('data/face_data_metadata.csv')
        print(f"✅ Face metadata loaded: {len(face_metadata)} faces")
    else:
        print("⚠️ Face metadata not found")
except Exception as e:
    print(f"⚠️ Error loading face metadata: {e}")

class HybridEngagementAnalyzer:
    """
    Hybrid system combining Face API and CNN predictions
    """
    
    def __init__(self, face_api_weight=0.7, cnn_weight=0.3):
        self.face_api_weight = face_api_weight
        self.cnn_weight = cnn_weight
        self.emotion_weights = {
            'angry': -0.10, 'disgusted': -0.05, 'fearful': -0.20,
            'sad': -0.05, 'happy': 0.10, 'surprised': 0.15, 'neutral': 0.30
        }
        self.emotion_labels = ['angry', 'disgusted', 'fearful', 'happy', 'sad', 'surprised', 'neutral']
    
    def predict_emotion_cnn(self, face_image_path):
        """Predict emotion using CNN model"""
        if fine_tuned_model is None or not os.path.exists(face_image_path):
            return None
            
        try:
            # Load and preprocess image
            img = cv2.imread(face_image_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                return None
                
            img_resized = cv2.resize(img, (48, 48))
            img_normalized = img_resized.reshape(1, 48, 48, 1) / 255.0
            
            # Predict
            prediction = fine_tuned_model.predict(img_normalized, verbose=0)[0]
            return dict(zip(self.emotion_labels, prediction))
            
        except Exception as e:
            print(f"⚠️ CNN prediction error for {face_image_path}: {e}")
            return None
    
    def calculate_engagement_hybrid(self, face_api_emotions, cnn_emotions=None):
        """Calculate engagement using hybrid approach"""
        
        # Face API engagement score
        face_api_score = sum(
            self.emotion_weights.get(emotion, 0) * prob 
            for emotion, prob in face_api_emotions.items()
        )
        
        # CNN engagement score (if available)
        cnn_score = None
        if cnn_emotions:
            cnn_score = sum(
                self.emotion_weights.get(emotion, 0) * prob 
                for emotion, prob in cnn_emotions.items()
            )
            
            # Weighted combination
            hybrid_score = (self.face_api_weight * face_api_score + 
                          self.cnn_weight * cnn_score)
        else:
            hybrid_score = face_api_score
        
        return {
            'face_api_score': face_api_score,
            'cnn_score': cnn_score,
            'hybrid_score': hybrid_score
        }
    
    def classify_engagement_level(self, score):
        """Classify engagement level based on score"""
        if score > 0.14:
            return "Highly Engaged"
        elif 0.10 <= score <= 0.14:
            return "Engaged"
        else:
            return "Disengaged"

# Initialize hybrid analyzer
hybrid_analyzer = HybridEngagementAnalyzer()

print("\n🔍 RUNNING HYBRID ANALYSIS...")

# Analyze each record
hybrid_results = []
cnn_predictions_count = 0

for idx, row in baseline_data.iterrows():
    # Face API emotions
    face_api_emotions = {
        'neutral': row['neutral'], 'happy': row['happy'], 'sad': row['sad'],
        'angry': row['angry'], 'fearful': row['fearful'], 
        'disgusted': row['disgusted'], 'surprised': row['surprised']
    }
    
    # Find corresponding CNN prediction (if available)
    cnn_emotions = None
    corresponding_face_path = None
    
    if face_metadata is not None:
        # Try to find matching face data
        matching_faces = face_metadata[
            (face_metadata['user_id'] == row['user_id'])
        ]
        
        if len(matching_faces) > 0:
            # Use first matching face
            face_row = matching_faces.iloc[0]
            corresponding_face_path = face_row['face_path']
            
            if os.path.exists(corresponding_face_path):
                cnn_emotions = hybrid_analyzer.predict_emotion_cnn(corresponding_face_path)
                if cnn_emotions:
                    cnn_predictions_count += 1
    
    # Calculate hybrid engagement
    engagement_scores = hybrid_analyzer.calculate_engagement_hybrid(
        face_api_emotions, cnn_emotions
    )
    
    # Classify engagement levels
    face_api_level = hybrid_analyzer.classify_engagement_level(engagement_scores['face_api_score'])
    hybrid_level = hybrid_analyzer.classify_engagement_level(engagement_scores['hybrid_score'])
    
    hybrid_results.append({
        'user_id': row['user_id'],
        'page': row.get('page', 'unknown'),
        'face_api_score': engagement_scores['face_api_score'],
        'cnn_score': engagement_scores['cnn_score'],
        'hybrid_score': engagement_scores['hybrid_score'],
        'face_api_level': face_api_level,
        'hybrid_level': hybrid_level,
        'has_cnn_prediction': cnn_emotions is not None,
        'face_path': corresponding_face_path
    })
    
    if (idx + 1) % 20 == 0:
        print(f"   Processed {idx + 1}/{len(baseline_data)} records...")

# Convert to DataFrame
hybrid_df = pd.DataFrame(hybrid_results)

print(f"\n✅ Hybrid analysis complete!")
print(f"📊 Processed {len(hybrid_df)} records")
print(f"🤖 CNN predictions available for {cnn_predictions_count} records ({cnn_predictions_count/len(hybrid_df)*100:.1f}%)")

# Comparison analysis
print(f"\n📈 COMPARISON ANALYSIS:")

# Engagement level comparison
face_api_dist = hybrid_df['face_api_level'].value_counts(normalize=True) * 100
hybrid_dist = hybrid_df['hybrid_level'].value_counts(normalize=True) * 100

print(f"\n🎯 Engagement Level Distribution:")
print(f"{'Level':<15} {'Face API (%)':<12} {'Hybrid (%)':<12} {'Difference':<12}")
print("-" * 55)

for level in ['Disengaged', 'Engaged', 'Highly Engaged']:
    face_api_pct = face_api_dist.get(level, 0)
    hybrid_pct = hybrid_dist.get(level, 0)
    diff = hybrid_pct - face_api_pct
    print(f"{level:<15} {face_api_pct:<12.1f} {hybrid_pct:<12.1f} {diff:+.1f}")

# Correlation analysis (for records with both predictions)
records_with_both = hybrid_df[hybrid_df['has_cnn_prediction'] == True]
if len(records_with_both) > 1:
    correlation = records_with_both['face_api_score'].corr(records_with_both['hybrid_score'])
    print(f"\n🔗 Score Correlation (records with CNN): {correlation:.4f}")
else:
    correlation = hybrid_df['face_api_score'].corr(hybrid_df['hybrid_score'])
    print(f"\n🔗 Overall Score Correlation: {correlation:.4f}")

# Visualizations
print(f"\n🎨 Creating comparison visualizations...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Score comparison scatter plot
axes[0,0].scatter(hybrid_df['face_api_score'], hybrid_df['hybrid_score'], 
                  alpha=0.6, color='blue')
axes[0,0].plot([-0.3, 0.3], [-0.3, 0.3], 'r--', label='Perfect Agreement')
axes[0,0].set_xlabel('Face API Score')
axes[0,0].set_ylabel('Hybrid Score')
axes[0,0].set_title('Face API vs Hybrid Engagement Scores')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# 2. Level distribution comparison
levels = ['Disengaged', 'Engaged', 'Highly Engaged']
x = np.arange(len(levels))
width = 0.35

face_api_counts = [face_api_dist.get(level, 0) for level in levels]
hybrid_counts = [hybrid_dist.get(level, 0) for level in levels]

axes[0,1].bar(x - width/2, face_api_counts, width, label='Face API', alpha=0.8)
axes[0,1].bar(x + width/2, hybrid_counts, width, label='Hybrid', alpha=0.8)
axes[0,1].set_xlabel('Engagement Level')
axes[0,1].set_ylabel('Percentage (%)')
axes[0,1].set_title('Engagement Level Distribution Comparison')
axes[0,1].set_xticks(x)
axes[0,1].set_xticklabels(levels)
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# 3. Per user comparison
user_comparison = hybrid_df.groupby('user_id').agg({
    'face_api_score': 'mean',
    'hybrid_score': 'mean'
}).reset_index()

axes[1,0].scatter(user_comparison['face_api_score'], user_comparison['hybrid_score'])
for i, user_id in enumerate(user_comparison['user_id']):
    axes[1,0].annotate(f'U{user_id}', 
                       (user_comparison.iloc[i]['face_api_score'], 
                        user_comparison.iloc[i]['hybrid_score']))
axes[1,0].plot([-0.3, 0.3], [-0.3, 0.3], 'r--', alpha=0.7)
axes[1,0].set_xlabel('Face API Average Score')
axes[1,0].set_ylabel('Hybrid Average Score')
axes[1,0].set_title('Per-User Average Engagement Comparison')
axes[1,0].grid(True, alpha=0.3)

# 4. CNN prediction coverage
coverage_data = {
    'With CNN': cnn_predictions_count,
    'Face API Only': len(hybrid_df) - cnn_predictions_count
}

axes[1,1].pie(coverage_data.values(), labels=coverage_data.keys(), autopct='%1.1f%%')
axes[1,1].set_title('CNN Prediction Coverage')

plt.tight_layout()
plt.savefig('results/hybrid_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Visualizations saved to results/hybrid_comparison.png")

# Save hybrid results
hybrid_df.to_csv('results/hybrid_analysis_results.csv', index=False)
print("💾 Hybrid results saved to results/hybrid_analysis_results.csv")

# Final summary
print(f"\n" + "="*70)
print("📋 FINAL HYBRID ANALYSIS SUMMARY")
print("="*70)

print(f"🔍 METHODOLOGY:")
print(f"   Face API Weight: {hybrid_analyzer.face_api_weight:.1f}")
print(f"   CNN Weight: {hybrid_analyzer.cnn_weight:.1f}")
print(f"   CNN Model: {'Fine-tuned' if 'fine_tuned_model.h5' in str(fine_tuned_model) else 'Pre-trained'}")

print(f"\n📊 PERFORMANCE METRICS:")
print(f"   Total Records: {len(hybrid_df):,}")
print(f"   Users: {hybrid_df['user_id'].nunique()}")
print(f"   CNN Coverage: {cnn_predictions_count}/{len(hybrid_df)} ({cnn_predictions_count/len(hybrid_df)*100:.1f}%)")
print(f"   Score Correlation: {correlation:.4f}")

print(f"\n🎯 ENGAGEMENT INSIGHTS:")
print(f"   Most Common Level (Face API): {face_api_dist.idxmax()} ({face_api_dist.max():.1f}%)")
print(f"   Most Common Level (Hybrid): {hybrid_dist.idxmax()} ({hybrid_dist.max():.1f}%)")

# Top users
user_insights = hybrid_df.groupby('user_id').agg({
    'hybrid_score': 'mean',
    'hybrid_level': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'Unknown'
}).round(4)

print(f"\n👥 TOP ENGAGED USERS (Hybrid Score):")
top_users = user_insights.sort_values('hybrid_score', ascending=False).head(3)
for user_id, data in top_users.iterrows():
    print(f"   User {user_id}: {data['hybrid_score']:.4f} ({data['hybrid_level']})")

print(f"\n🎉 HYBRID ANALYSIS COMPLETE!")
print(f"📁 All results saved to 'results/' directory")

# Export final summary
final_summary = {
    'methodology': f'Hybrid Face API ({hybrid_analyzer.face_api_weight}) + CNN ({hybrid_analyzer.cnn_weight})',
    'total_records': len(hybrid_df),
    'users': int(hybrid_df['user_id'].nunique()),
    'cnn_coverage': f"{cnn_predictions_count/len(hybrid_df)*100:.1f}%",
    'score_correlation': float(correlation),
    'engagement_distribution': {
        'face_api': face_api_dist.to_dict(),
        'hybrid': hybrid_dist.to_dict()
    },
    'top_users': top_users.to_dict() if len(top_users) > 0 else {},
    'estimated_improvement': 'Hybrid system provides more robust engagement detection'
}

with open('results/final_hybrid_summary.json', 'w') as f:
    json.dump(final_summary, f, indent=2, default=str)

print(f"💾 Final summary saved to results/final_hybrid_summary.json")

print(f"\n🎊 CONGRATULATIONS! Hybrid engagement detection system completed!")